In [10]:
%run summarizationBackbone.ipynb

Loading Big File...
Finished Loading Big File.


In [11]:
from preprocessing_algorithms import *
from tqdm import tqdm

In [12]:
ds = pd.read_csv("Duc_dataset_first_ref_summary.csv")

<h1>Summarization Settings

In [5]:
articleCol = "Original Article"
summaryCol = "Original Summary"
start = 0
end = len(ds)
percentage = 5
lemmatization = False
remove_stopwords = False
outFileName = "DUC_1st_5Sent_lemT_swT.csv"
useWeights = True
weights = {"tm":0.92,"luhn":1,"lsa":0.44,"tr":0.84,"lex":0.88,"lda":0.8}

In [6]:
Algorithms = ["Tm","Lex","Luhn","Lsa","Tr","LDA"]
lstCols = get_combinations(Algorithms)
cols = ["Original Article","Original Summary"] + lstCols
df3 = pd.DataFrame(columns = cols)
df3.to_csv(outFileName, index=False)

In [7]:
summarizedDataset = []
for i in tqdm(range(start,end,1)):
    try:
        sentences, filtered_sentences = preprocessing_text_with_spacy(ds[articleCol][i],lemmatization,remove_stopwords)
        df = buildDF(filtered_sentences, sentences, useWeights, weights)
        
        summarizedRow = {"Original Article": ds[articleCol][i],"Original Summary": ds[summaryCol][i]}
        for key in df.keys():
            element = summarizeWith(sentences, df, key, percentage)
            summarizedRow[key] = element
        pd.DataFrame.from_dict(summarizedRow, orient='index').T.to_csv(outFileName,mode='a',header=False,index=False)
    except Exception as e:
        print("Error",i, e)

100%|██████████| 10/10 [07:17<00:00, 43.71s/it]


In [36]:
def predict(text, percentage):
    try:
        sentences, filtered_sentences = preprocessing_text_with_spacy(text,True,False)
        df = buildDF(filtered_sentences, sentences)
        summarizedRow = {"Original Article": ds[articleCol][i]}
        for key in df.keys():
            element = summarizeWith(sentences, df, key, percentage)
            summarizedRow[key] = element
        return summarizedRow
    except Exception as e:
        print("Error",i, e)